In [0]:
df = (spark.readStream
           .format("cloudFiles")
           .option("cloudFiles.format", "csv")
           .option("cloudFiles.inferColumnTypes", "true")
           .load("s3://it-gwsc-dvp-prd-s3/prod/trends_era5/daily_summary/"))

df.writeStream.format("delta").option("checkpointLocation", "dbfs:/checkpoints/mytable").start("dbfs:/output/mytable")


import boto3
import subprocess
import os
d
bucket_name = 'it-gwsc-dvp-prd-s3'
prefix = 'prod/trends_era5/daily_summary/'

s3 = boto3.client("s3")
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

for page in pages:
    for obj in page.get("Contents", []):
        print(obj["Key"])
        break
    break


source_folder = '/Volumes/aer-processed/era5/daily_summary'
first_file = next(iter(os.scandir(source_folder)))
print(first_file)

#spark.sql("""
#    OPTIMIZE `era5-daily-data`.bronze_dev.era5_inventory_table
#    ZORDER BY (date_modified_in_s3);
#""")

latest_date_df = spark.sql("""
    SELECT MAX(date_modified_in_s3) AS max_date
    FROM `era5-daily-data`.bronze_dev.era5_inventory_table
""")
display(latest_date_df)

s3 = boto3.client('s3')
paginator = s3.get_paginator()